In [1]:
# 安装模型后，restart一下，然后不要再运行这一行了
! python -m spacy download zh_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.9/237.9 kB 5.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 测试一下spacy可以正常使用
import spacy
nlp = spacy.load("zh_core_web_trf")
import zh_core_web_trf
nlp = zh_core_web_trf.load()
doc = nlp("这是一个用于示例的句子。")
print([(w.text, w.pos_) for w in doc])

[('这是', 'PRON'), ('一个', 'NUM'), ('用于', 'VERB'), ('示例', 'NOUN'), ('的', 'PART'), ('句子', 'NOUN'), ('。', 'PUNCT')]


In [3]:
# === 1) 路径设置（按你的要求） ===
DATA_FILE_PATH = "/kaggle/input/nssfc-2025/nssfc.csv"


# === 2) 读取数据 ===
import re
import pandas as pd

try:
    df = pd.read_csv(DATA_FILE_PATH)
except UnicodeDecodeError:
    # 若编码报错，回退到常见中文编码
    df = pd.read_csv(DATA_FILE_PATH, encoding="utf-8-sig")

# 列名检查
if "课题名称" not in df.columns:
    raise ValueError("找不到列：【课题名称】。请确认输入文件包含该列。现有列为："
                     + ", ".join(map(str, df.columns.tolist())))

# === 3) 分词（向量化加速） ===
# 说明：保留与原代码一致的规则——过滤长度为 1 的 token
def _token_filter(token_text: str) -> bool:
    return len(token_text.strip()) > 1

# === 清洗函数：去除所有中英文引号 ===
def clean_quotes(text: str) -> str:
    if pd.isna(text):
        return ""
    # 去除常见中英文引号：“ ” ‘ ’ " '
    return re.sub(r"[\"\'“”‘’]", "", str(text))

# === 读取并清洗 ===
df["课题名称"] = df["课题名称"].astype(str).apply(clean_quotes)

# === 提取文本列表 ===
texts = df["课题名称"].tolist()

segmented = []
# 使用 nlp.pipe 比逐行 nlp 调用更快
for doc in nlp.pipe(texts, batch_size=128):
    toks = [t.text.strip() for t in doc if _token_filter(t.text)]
    segmented.append(" ".join(toks))

df["课题名称分词"] = segmented

# === 4) 保存 ===
OUT_PATH = "nssfc-fenci.csv"
df.to_csv(OUT_PATH, index=False)
print(f"已保存：{OUT_PATH}")
print(df.head(3)[["课题名称", "课题名称分词"]])

已保存：nssfc-fenci.csv
                     课题名称                     课题名称分词
0     习近平总书记关于民营经济的重要论述研究  习近平 总书记 关于 民营 经济 重要 论述 研究
1      习近平总书记关于共同体的重要论述研究    习近平 总书记 关于 共同体 重要 论述 研究
2  《资本论》及其手稿中生产力理论的当代价值研究  资本论 及其 手稿 生产力 理论 当代 价值 研究
